# **Colab Pro notebook from https://github.com/TheLastBen/fast-stable-diffusion** *Alternatives : [Paperspace](https://console.paperspace.com/github/TheLastBen/PPS?machine=Free-GPU)*

In [3]:
#@markdown # Connect Google Drive
from google.colab import drive
from IPython.display import clear_output
import ipywidgets as widgets
import os

def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
Shared_Drive = "" #@param {type:"string"}
#@markdown - Leave empty if you're not using a shared drive

print("[0;33mConnecting...")
drive.mount('/content/gdrive')

if Shared_Drive!="" and os.path.exists("/content/gdrive/Shareddrives"):
  mainpth="Shareddrives/"+Shared_Drive
else:
  mainpth="MyDrive"

clear_output()
inf('\u2714 Done','success', '50px')

#@markdown ---

Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

In [4]:
#@markdown # Install/Update AUTOMATIC1111 repo
from IPython.utils import capture
from IPython.display import clear_output
from subprocess import getoutput
import ipywidgets as widgets
import sys
import fileinput
import os
import time
import base64
import requests
from urllib.request import urlopen, Request
from urllib.parse import urlparse, parse_qs, unquote
from tqdm import tqdm
import six


blsaphemy=base64.b64decode(("ZWJ1aQ==").encode('ascii')).decode('ascii')

if not os.path.exists("/content/gdrive"):
  print('[1;31mGdrive not connected, using temporary colab storage ...')
  time.sleep(4)
  mainpth="MyDrive"
  !mkdir -p /content/gdrive/$mainpth
  Shared_Drive=""

if Shared_Drive!="" and not os.path.exists("/content/gdrive/Shareddrives"):
  print('[1;31mShared drive not detected, using default MyDrive')
  mainpth="MyDrive"

with capture.capture_output() as cap:
  def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
  fgitclone = "git clone --depth 1"
  %mkdir -p /content/gdrive/$mainpth/sd
  %cd /content/gdrive/$mainpth/sd
  !git clone -q --branch master https://github.com/AUTOMATIC1111/stable-diffusion-w$blsaphemy
  !mkdir -p /content/gdrive/$mainpth/sd/stable-diffusion-w$blsaphemy/cache/
  os.environ['TRANSFORMERS_CACHE']=f"/content/gdrive/{mainpth}/sd/stable-diffusion-w"+blsaphemy+"/cache"
  os.environ['TORCH_HOME'] = f"/content/gdrive/{mainpth}/sd/stable-diffusion-w"+blsaphemy+"/cache"
  !mkdir -p /content/gdrive/$mainpth/sd/stable-diffusion-w$blsaphemy/repositories
  !git clone https://github.com/AUTOMATIC1111/stable-diffusion-w$blsaphemy-assets /content/gdrive/$mainpth/sd/stable-diffusion-w$blsaphemy/repositories/stable-diffusion-webui-assets

with capture.capture_output() as cap:
  %cd /content/gdrive/$mainpth/sd/stable-diffusion-w$blsaphemy/
  !git reset --hard
  !git checkout master
  time.sleep(1)
  !rm webui.sh
  !git pull
clear_output()
inf('\u2714 Done','success', '50px')

#@markdown ---

Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

In [5]:
#@markdown # Requirements

print('[1;32mInstalling requirements...')

with capture.capture_output() as cap:
  %cd /content/
  !wget -q -i https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dependencies/A1111.txt
  !dpkg -i *.deb
  if not os.path.exists('/content/gdrive/'+mainpth+'/sd/stablediffusion'):
    !tar -C /content/gdrive/$mainpth --zstd -xf sd_mrep.tar.zst
  !tar -C / --zstd -xf gcolabdeps.tar.zst
  !rm *.deb | rm *.zst | rm *.txt
  if not os.path.exists('gdrive/'+mainpth+'/sd/libtcmalloc/libtcmalloc_minimal.so.4'):
    %env CXXFLAGS=-std=c++14
    !wget -q https://github.com/gperftools/gperftools/releases/download/gperftools-2.5/gperftools-2.5.tar.gz && tar zxf gperftools-2.5.tar.gz && mv gperftools-2.5 gperftools
    !wget -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/AUTOMATIC1111_files/Patch
    %cd /content/gperftools
    !patch -p1 < /content/Patch
    !./configure --enable-minimal --enable-libunwind --enable-frame-pointers --enable-dynamic-sized-delete-support --enable-sized-delete --enable-emergency-malloc; make -j4
    !mkdir -p /content/gdrive/$mainpth/sd/libtcmalloc && cp .libs/libtcmalloc*.so* /content/gdrive/$mainpth/sd/libtcmalloc
    %env LD_PRELOAD=/content/gdrive/$mainpth/sd/libtcmalloc/libtcmalloc_minimal.so.4
    %cd /content
    !rm *.tar.gz Patch && rm -r /content/gperftools
  else:
    %env LD_PRELOAD=/content/gdrive/$mainpth/sd/libtcmalloc/libtcmalloc_minimal.so.4

  !pip install controlnet_aux -qq --no-deps
  !rm -r /usr/local/lib/python3.10/dist-packages/google/_upb
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
  os.environ['PYTHONWARNINGS'] = 'ignore'
  !sed -i 's@text = _formatwarnmsg(msg)@text =\"\"@g' /usr/lib/python3.10/warnings.py

clear_output()
inf('\u2714 Done','success', '50px')

#@markdown ---

Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

In [6]:
#@markdown # Model Download/Load

import gdown
from gdown.download import get_url_from_gdrive_confirmation
import re
from urllib.parse import urlparse, parse_qs, unquote
import os
from tqdm import tqdm
from urllib.request import Request, urlopen

# Variáveis principais
mainpth = 'MyDrive'  # Nome do diretório principal no Google Drive
blsaphemy = ''  # Se necessário, ajuste ou deixe vazio
Use_Temp_Storage = False  #@param {type:"boolean"}

# Opções de modelo
Model_Version = "SDXL"  #@param ["SDXL", "1.5", "v1.5 Inpainting", "V2.1-768px"]
PATH_to_MODEL = ""  #@param {type:"string"}
MODEL_LINK = "https://huggingface.co/SG161222/Realistic_Vision_V6.0_B1_noVAE/resolve/main/Realistic_Vision_V6.0_NV_B1.safetensors"  #@param {type:"string"}

# Função para identificar a origem do link
def getsrc(url):
    parsed_url = urlparse(url)
    if parsed_url.netloc == 'civitai.com':
        return 'civitai'
    elif parsed_url.netloc == 'drive.google.com':
        return 'gdrive'
    elif parsed_url.netloc == 'huggingface.co':
        return 'huggingface'
    else:
        return 'others'

src = getsrc(MODEL_LINK)

# Função para obter o nome do arquivo do link
def get_name(url, gdrive):
    if not gdrive:
        response = urlopen(url)
        redirected_url = response.geturl()
        return os.path.basename(redirected_url)
    else:
        lnk = f"https://drive.google.com/uc?id={url.split('/d/')[1].split('/view')[0]}&export=download"
        response = requests.get(lnk, stream=True)
        content_disposition = response.headers.get("content-disposition", "")
        match = re.search(r'filename="(.+)"', content_disposition)
        return match.group(1) if match else "model.safetensors"

# Função para criar diretórios
def create_dirs(path):
    os.makedirs(path, exist_ok=True)

# Função para download do modelo
def dwn(url, dst, msg):
    req = Request(url, headers={"User-Agent": "torch.hub"})
    u = urlopen(req)

    # Corrigir nome do arquivo longo
    dst = os.path.join(os.path.dirname(dst), "model.safetensors")  # Define um nome fixo

    file_size = int(u.headers["Content-Length"])

    with tqdm(total=file_size, unit='B', unit_scale=True, desc=msg) as pbar:
        with open(dst, "wb") as f:
            while True:
                buffer = u.read(8192)
                if not buffer:
                    break
                f.write(buffer)
                pbar.update(len(buffer))
    print(f"✔ Model downloaded to {dst}")

# Função para selecionar o modelo correto
def sdmdls(ver, Use_Temp_Storage):
    base_path = '/content/temp_models' if Use_Temp_Storage else f'/content/gdrive/{mainpth}/sd/stable-diffusion-w{blsaphemy}/models/Stable-diffusion'
    create_dirs(base_path)

    if ver == '1.5':
        model = f"{base_path}/v1-5-pruned-emaonly.safetensors"
        link = "https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors"
    elif ver == 'V2.1-768px':
        model = f"{base_path}/v2-1_768-ema-pruned.safetensors"
        link = "https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors"
    elif ver == 'v1.5 Inpainting':
        model = f"{base_path}/sd-v1-5-inpainting.ckpt"
        link = "https://huggingface.co/runwayml/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt"
    elif ver == 'SDXL':
        model = f"{base_path}/sd_xl_base_1.0.safetensors"
        link = "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors"
    else:
        raise ValueError("Invalid Model Version")

    if not os.path.exists(model):
        dwn(link, model, f"Downloading {ver}")
    return model

# Lógica principal para baixar ou carregar o modelo
if PATH_to_MODEL:
    if os.path.exists(PATH_to_MODEL):
        print(f"✔ Using custom model from {PATH_to_MODEL}")
        model = PATH_to_MODEL
    else:
        raise FileNotFoundError(f"Path {PATH_to_MODEL} does not exist!")
elif MODEL_LINK:
    base_path = '/content/temp_models' if Use_Temp_Storage else f'/content/gdrive/{mainpth}/sd/stable-diffusion-w{blsaphemy}/models/Stable-diffusion'
    create_dirs(base_path)
    modelname = get_name(MODEL_LINK, src == 'gdrive')
    model = f"{base_path}/{modelname}"
    if not os.path.exists(model):
        dwn(MODEL_LINK, model, "Downloading custom model")
    print(f"✔ Model downloaded to {model}")
else:
    model = sdmdls(Model_Version, Use_Temp_Storage)

print(f"✔ Final model path: {model}")


✔ Model downloaded to /content/gdrive/MyDrive/sd/stable-diffusion-w/models/Stable-diffusion/model.safetensors
✔ Model downloaded to /content/gdrive/MyDrive/sd/stable-diffusion-w/models/Stable-diffusion/5d814d2f9c489f9d3bffc56126a66c182eee7729bffacf71c048260aa366c269?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Realistic_Vision_V6.0_NV_B1.safetensors%3B+filename%3D%22Realistic_Vision_V6.0_NV_B1.safetensors%22%3B&Expires=1735352034&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNTM1MjAzNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2Q4Lzk1L2Q4OTUyODdjY2Y3MzlkMzkzYTg0MzkyZDFjMjVlYzhjMmM1MmI3MGQwMTJiMzJmZGZjYThjYTQ1YTRiZTBhMzIvNWQ4MTRkMmY5YzQ4OWY5ZDNiZmZjNTYxMjZhNjZjMTgyZWVlNzcyOWJmZmFjZjcxYzA0ODI2MGFhMzY2YzI2OT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=URNc30Vx4gUnYZvnAYVEoGH7%7ES36Ck4iie1h2ESD06NJHZRKlVDS3ujPRkFQ0%7EkaSruPX9o5tWNLZCXkF6u0MzzOrQkoBfXbf8V2TFaEqlyXsMmFlHXKWTJlT1gjZeMX2bWaeWVArMnkUMR

In [7]:
import os

# Parâmetro para o link do LoRA
LoRA_LINK = "https://huggingface.co/username/lora_model/resolve/main/lora_file.safetensors"
 # Insira aqui o link direto do LoRA

# Verifica se o link foi fornecido
if LoRA_LINK == "":
    print("✔ Nothing to do")
else:
    # Cria o diretório necessário
    lora_dir = f'/content/gdrive/{mainpth}/sd/stable-diffusion-w{blsaphemy}/models/Lora'
    os.makedirs(lora_dir, exist_ok=True)

    # Define o caminho do arquivo e baixa o LoRA
    lora_file = os.path.join(lora_dir, "lora.safetensors")
    if not os.path.exists(lora_file):
        print(f"⬇ Downloading LoRA from {LoRA_LINK}")
        !gdown --fuzzy -O {lora_file} {LoRA_LINK}
    else:
        print(f"✔ LoRA already exists at {lora_file}")


⬇ Downloading LoRA from https://huggingface.co/username/lora_model/resolve/main/lora_file.safetensors
Downloading...
From: https://huggingface.co/username/lora_model/resolve/main/lora_file.safetensors
To: /content/gdrive/MyDrive/sd/stable-diffusion-w/models/Lora/lora.safetensors
100% 29.0/29.0 [00:00<00:00, 113kB/s]


In [9]:
import os
from IPython.utils.capture import capture_output

# Variáveis principais
mainpth = 'MyDrive'  # Certifique-se de ajustar para seu diretório principal no Google Drive
blsaphemy = ''  # Ajuste ou deixe vazio

# Lógica para o ControlNet
controlnet_dir = f'/content/gdrive/{mainpth}/sd/stable-diffusion-w{blsaphemy}/extensions/controlnet'

# Criar diretório, se não existir
if not os.path.exists(controlnet_dir):
    os.makedirs(controlnet_dir, exist_ok=True)
    print(f"✔ ControlNet directory created: {controlnet_dir}")

# Baixar/atualizar extensão ControlNet
with capture_output() as cap:
    # Mover para o diretório da extensão
    get_ipython().run_line_magic('cd', f'/content/gdrive/{mainpth}/sd/stable-diffusion-w{blsaphemy}/extensions')

    # Clonar ou atualizar o repositório ControlNet
    if not os.path.exists(f'{controlnet_dir}/.git'):
        print("⬇ Cloning ControlNet extension...")
        get_ipython().system('git clone https://github.com/Mikubill/sd-webui-controlnet.git controlnet')
    else:
        print("🔄 Updating ControlNet extension...")
        get_ipython().run_line_magic('cd', controlnet_dir)
        get_ipython().system('git pull')

# Retornar ao diretório padrão
get_ipython().run_line_magic('cd', '/content')

print("✔ ControlNet extension setup completed.")


/content
✔ ControlNet extension setup completed.


In [ ]:
# ================================
# 1. Remover Bibliotecas Problemáticas
# ================================
!pip uninstall -y pydantic transformers wandb pytorch-lightning torchmetrics accelerate fastapi starlette

# ================================
# 2. Instalar Versões Compatíveis
# ================================
!pip install "pydantic<2.0.0" \
"transformers<4.31.0" \
"wandb<0.15.0" \
"pytorch-lightning<1.10" \
"torchmetrics<1.0.0" \
"accelerate<1.0.0" \
"fastapi<0.89.0" \
"starlette<0.23.0"

# ================================
# 3. Instalar Dependências do Projeto
# ================================
!pip install gradio==3.41.2 \
altair \
aiohttp==3.8.1 \
scikit-image>=0.19 \
torchvision \
torchaudio \
numpy==1.23.5 \
safetensors>=0.3.1

# ================================
# 4. Clonar e Configurar o Repositório do Stable Diffusion
# ================================
import os
if not os.path.exists("/content/stable-diffusion-webui"):
    !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git /content/stable-diffusion-webui

%cd /content/stable-diffusion-webui

# Atualizar o repositório para a versão mais recente
!git pull

# ================================
# 5. Instalar Requisitos do Stable Diffusion
# ================================
!pip install -r requirements.txt

# ================================
# 6. Iniciar o Stable Diffusion WebUI
# ================================
print("🔄 Iniciando o Stable Diffusion WebUI...")
!python launch.py --share


Found existing installation: pydantic 1.10.19
Uninstalling pydantic-1.10.19:
  Successfully uninstalled pydantic-1.10.19
Found existing installation: transformers 4.30.2
Uninstalling transformers-4.30.2:
  Successfully uninstalled transformers-4.30.2
Found existing installation: wandb 0.19.1
Uninstalling wandb-0.19.1:
  Successfully uninstalled wandb-0.19.1
Found existing installation: pytorch-lightning 1.9.4
Uninstalling pytorch-lightning-1.9.4:
  Successfully uninstalled pytorch-lightning-1.9.4
Found existing installation: torchmetrics 0.10.3
Uninstalling torchmetrics-0.10.3:
  Successfully uninstalled torchmetrics-0.10.3
Found existing installation: accelerate 0.21.0
Uninstalling accelerate-0.21.0:
  Successfully uninstalled accelerate-0.21.0
Found existing installation: fastapi 0.94.0
Uninstalling fastapi-0.94.0:
  Successfully uninstalled fastapi-0.94.0
Found existing installation: starlette 0.26.1
Uninstalling starlette-0.26.1:
  Successfully uninstalled starlette-0.26.1
  Using 

In [16]:
# Clonar o repositório (somente se não existir)
import os
if not os.path.exists("/content/stable-diffusion-webui"):
    !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git /content/stable-diffusion-webui

# Navegar para o diretório do Stable Diffusion
%cd /content/stable-diffusion-webui

# Atualizar pacotes problemáticos
!pip install -U lightning==2.0.0 pydantic==1.10.12
!pip install -U transformers==4.30.2 accelerate wandb
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Instalar dependências do Stable Diffusion
!pip install -r requirements.txt

# Iniciar o Stable Diffusion WebUI
print("🔄 Starting Stable-Diffusion...")
!python launch.py --share


/content/stable-diffusion-webui
  Using cached pydantic-1.10.12-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (149 kB)
  Using cached fastapi-0.88.0-py3-none-any.whl.metadata (24 kB)
  Using cached starlette-0.22.0-py3-none-any.whl.metadata (5.8 kB)
Using cached pydantic-1.10.12-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
Using cached fastapi-0.88.0-py3-none-any.whl (55 kB)
Using cached starlette-0.22.0-py3-none-any.whl (64 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.19
    Uninstalling pydantic-1.10.19:
      Successfully uninstalled pydantic-1.10.19
  Attempting uninstall: starlette
    Found existing installation: starlette 0.26.1
    Uninstalling starlette-0.26.1:
      Successfully uninstalled starlette-0.26.1
  Attempting uninstall: fastapi
    Found existing installation: fastapi 0.94.0
    Uninstalling fastapi-0.94.0:
      Successfully uninstalled fastapi-0.94.0
ERROR: pip's dependency resolv

In [20]:
# Instalar a biblioteca pyngrok se não estiver instalada
try:
    from pyngrok import ngrok, conf
except ImportError:
    !pip install pyngrok
    from pyngrok import ngrok, conf

import os

# Variáveis do usuário
Use_Cloudflare_Tunnel = False  #@param {type:"boolean"}
Ngrok_token = ""  #@param {type:"string"}
User = ""  #@param {type:"string"}
Password = ""  #@param {type:"string"}

# Configuração do túnel com Ngrok
if Ngrok_token:
    conf.get_default().auth_token = Ngrok_token
    public_url = ngrok.connect(7860, "http")
    print(f"✔ Public URL: {public_url}")
elif Use_Cloudflare_Tunnel:
    print("✔ Using Cloudflare Tunnel...")
else:
    print("✘ No tunnel configured. Access only locally or within Colab.")

# Configuração de autenticação do Gradio
if User and Password:
    os.environ["GRADIO_AUTH"] = f"{User}:{Password}"
    print("✔ Gradio interface secured with user authentication.")
else:
    print("✔ Gradio interface open (no authentication).")

# Iniciar o Stable-Diffusion WebUI
print("🔄 Starting Stable-Diffusion...")
!python launch.py --share


✘ No tunnel configured. Access only locally or within Colab.
✔ Gradio interface open (no authentication).
🔄 Starting Stable-Diffusion...
python3: can't open file '/content/gdrive/MyDrive/sd/stable-diffusion-w/launch.py': [Errno 2] No such file or directory
